# 创建一个Neural Network
Neural Network包含对数据执行操作的layer或者是module。<kbd>torch.nn</kbd>命名空间提供了所有的你需要创建你自己的neural network的模块。在Pytorch中，每一个Pytorch中的module都是<kbd>nn.Module</kbd>的子类。一个neural network是包括其它module(layers)的module。这种嵌套结构(nested structure)允许方便的创建和管理复杂的architecture。

在接下来的部分，我们将创建一个neural network去给FashionMNIST的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

## 选择Device进行训练
我们可以使用类似于GPU一样的hardware accelerator训练我们的model。如果hardware accelerator可用，我们就使用torch.cuda检查是否hardware accelerator可用，如果不可用我们就使用CPU。

In [3]:
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 定义Model类

我们定义我们的neural network,它是nn.Model的子类，然后通过\_\_init\_\_方法来初始化neural network layer。每个<kbd>nn.Module</kbd>子类通过forword方法实现关于input data上的操作。

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

创建一个NeuralNetwork实例，将这个模型放到指定的device上，输出model的结构。

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


我们将input data传入到model中。这样做会执行model的forward方法以及一些background operations。千万不要直接调用model.forword()方法。

调用model后会返回一个带有原始predicted value的10维tensor。我们通过将这个predicted value传入到实例化的<kbd>nn.Softmax</kbd> 模块中，得到prediction probability。

In [6]:
X=torch.rand(1,28,28,device=device)
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits)
y_pred=pred_probab.argmax(1)
print(f"Predicted class:{y_pred}")

Predicted class:tensor([1], device='cuda:0')


## Model Layers
现在我们来分析一下 FashionMNIST model。我们使用3张28x28 size的图片大小的sample minibatch，来看看当我们把它们输入进去network中时会发生说明情况。

In [7]:
input_image=torch.rand(3,28,28)
print(input_image.size())


torch.Size([3, 28, 28])


## nn.Flatten
我们初始化nn.Flatten layer将2D的大小为28x28 pixels的图片转换为连续的784个pixel value的数组。(将继续维持minibatch的维数(在dim=0处))

In [8]:
flatten=nn.Flatten()
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear
linear layer module 可以实现把使用所存的weight和bias的input data做一个linear transformation。

In [10]:
layer1=nn.Linear(in_features=28*28,out_features=20)
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
Non-Linear activation在model的input和output之间创建了一个复杂的mapping。他们在linear transformation后引入non-linearity,帮助neural networks学习到更多的情况。

在这个model中，我们在linear layer之间使用<kbd>nn.ReLU</kbd>，当然还有其它的activation可以在你的model中引入non-linearity。

In [11]:
print(f"Before ReLU:{hidden1}\n\n")
hidden1=nn.ReLU()(hidden1)
print(f"After ReLU:{hidden1}")


Before ReLU:tensor([[ 0.4980,  0.4183,  0.4729,  0.1953, -0.5796, -0.5525, -0.1373,  0.0244,
          0.4123, -0.4585,  0.2454,  0.2212, -0.0457, -0.0988,  0.0707, -0.3111,
         -0.9504, -0.4235, -0.1288, -0.1715],
        [ 0.0538,  0.2393,  0.0531, -0.2433, -0.7757,  0.0516, -0.0574, -0.0772,
          0.0229, -0.2277,  0.2100,  0.2677,  0.1691, -0.1607,  0.2274, -0.3940,
         -0.4249, -0.1693, -0.3229,  0.0180],
        [ 0.2991,  0.6463,  0.3149, -0.4555, -0.5128,  0.0071, -0.1014,  0.1511,
          0.2076,  0.0130,  0.4172,  0.1504, -0.1808, -0.7544,  0.6730, -0.1057,
         -0.9215, -0.5958, -0.3144,  0.0216]], grad_fn=<AddmmBackward>)


After ReLU:tensor([[0.4980, 0.4183, 0.4729, 0.1953, 0.0000, 0.0000, 0.0000, 0.0244, 0.4123,
         0.0000, 0.2454, 0.2212, 0.0000, 0.0000, 0.0707, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0538, 0.2393, 0.0531, 0.0000, 0.0000, 0.0516, 0.0000, 0.0000, 0.0229,
         0.0000, 0.2100, 0.2677, 0.1691, 0.0000, 0.2274,

## nn.Sequential
<kbd>nn.Sequential</kbd>是一个有序的module容器。data经过同样顺序的modules。你可以是哟红sequential container去把一个像是 seq_modules 的quick network整合到一起。

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
最后一层的neural network的linear layer返回logits-在\[-infty.+infty\]的原始值，将这些logits经过<kbd>nn.Softmax</kbd> 模块处理后，logits的范围会缩放至\[0,1\]之间，表示model对每一个class的predicted probabilites。dim参数表示的维数上数字之和必须等于1的维度。

In [13]:
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

## Model Parameters
神经网络内部许多层都是参数化的，即具有在训练期间优化的相关weight和bias。 子类nn.module会自动跟踪model object中定义的所有field，并使所有参数可以使用你的parameters()或named_parameters（）方法访问。

在这个例子中，我们遍历每个参数，并打印其大小和预览值。

In [14]:
print("Model structure:",model,"\n\n")

for name,param in model.named_parameters():
    print(f"layer:{name}| Size:{param.size()}|Values:{param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


layer:linear_relu_stack.0.weight| Size:torch.Size([512, 784])|Values:tensor([[-0.0180, -0.0004, -0.0056,  ..., -0.0180, -0.0015, -0.0194],
        [ 0.0022,  0.0048, -0.0076,  ..., -0.0220, -0.0009, -0.0261]],
       device='cuda:0', grad_fn=<SliceBackward>)

layer:linear_relu_stack.0.bias| Size:torch.Size([512])|Values:tensor([-0.0144, -0.0280], device='cuda:0', grad_fn=<SliceBackward>)

layer:linear_relu_stack.2.weight| Size:torch.Size([512, 512])|Values:tensor([[-0.0346,  0.0274, -0.0099,  ...,  0.0365, -0.0054,  0.0073],
        [ 0.0044,  0.0376, -0.0375,  ..., -0.0073, -0.0122, -0.0165]],
       device='cuda:0', grad_fn=<SliceBackw

## 扩展阅读
[torch.nn API](https://pytorch.org/docs/stable/nn.html)